In [ ]:
import mysql.connector
import mysql
import pandas as pd
import os
import numpy as np



In [90]:
file_path = os.path.expanduser("~/password/sql.txt")
file = open(file_path,'r')
password = file.readline()
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = "neuralnetwork"
    
)
mycursor = mydb.cursor()

In [91]:
def save_weight_db(file_path,weight_name):
    read = pd.read_csv(file_path,header = None)
    data = np.array(read)
    
    length = data.shape[1]
    mycursor = mydb.cursor()
    name = 'layer'+weight_name
    mycursor.execute(f"DROP TABLE {name}")
    mycursor.execute(f'''CREATE TABLE IF NOT EXISTS {name}(
                     weight_1 FLOAT)
                  ''')
    for i in range(1,length):
        mycursor.execute(f"alter table {name} add weight_{i+1} FLOAT ")
    

In [115]:
for name in os.listdir(os.path.expanduser('~/Tumor-Classification/Model_Weights')):
    # Open file
    file_path = os.path.join(os.path.expanduser('~/Tumor-Classification/Model_Weights'), name)
    split = str(name).split('.')
    save_weight_db(file_path,split[0])
    print(f'saved to {split[0]}')

    

saved to three_bias
saved to one_bias
saved to one_weights
saved to four_weights
saved to two_weights
saved to two_bias
saved to four_bias
saved to three_weights


In [114]:
def store_weight_db(file_path,weight_name):
    # name = 'layer'+weight_name
    # read = pd.read_csv(file_path,header=None)
    # data = np.array(read)
    # for row in data:
    #     for value in range(len(row)):
    #         command = f"INSERT INTO {name} (weight_{value+1}) VALUES ({row[value]})"
            
    #         mycursor.execute(command)
    name = 'layer' + weight_name
    read = pd.read_csv(file_path, header=None)
    data = np.array(read)
    
    for row in data:
        # Create a single SQL command that inserts all values from this row at once
        columns = ", ".join([f"weight_{value+1}" for value in range(len(row))])
        values = ", ".join([str(row[value]) for value in range(len(row))])
        
        command = f"INSERT INTO {name} ({columns}) VALUES ({values});"
        
        mycursor.execute(command)
        mydb.commit()
    
# store_weight_db(os.path.expanduser('~/Tumor-Classification/Model_Weights/three_weights.csv'),"three_weights")



In [116]:
for name in os.listdir(os.path.expanduser('~/Tumor-Classification/Model_Weights')):
    # Open file
    file_path = os.path.join(os.path.expanduser('~/Tumor-Classification/Model_Weights'), name)
    split = str(name).split('.')
    store_weight_db(file_path,split[0])
    print(f'stored to {split[0]}')

stored to three_bias
stored to one_bias
stored to one_weights
stored to four_weights
stored to two_weights
stored to two_bias
stored to four_bias
stored to three_weights


In [58]:
file = os.path.join(os.path.expanduser('~/Tumor-Classification/Model_Weights'), "one_weights.csv")
name = "one_weights"
store_weight_db(file,name)

In [79]:
mycursor.execute('delete from layerthree_weights')

In [ ]:
mycursor.execute("SELECT * FROM layerone_weights")

myresult = mycursor.fetchall()
data = np.array(myresult)
print(data)

In [87]:
def load_weights_db(network):
    for layer in network:
        if isinstance(layer,Dense):
            #weights
            name = layer.name
            mycursor.execute((f"SELECT * FROM layer{name}_weights"))
            myresult = mycursor.fetchall()
            weight = np.array(myresult)
            layer.weights = weight
            print("Loaded weights:", layer.name)
            #bias
            mycursor.execute((f"SELECT * FROM layer{name}_bias"))
            myresult = mycursor.fetchall()
            bias = np.array(myresult)
            layer.bias = bias
            print("Loaded bias:", layer.name)